In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_3.csv")

In [3]:
df

,timestamp_local,temp,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,430,1782.5,60.7,2.7,533.7,377.00,253.3
...,...,...,...,...,...,...,...,...,...
20973,23/06/2024 19:00,39.3,92,155.6,21.0,105.3,57.0,31.33,17.0
20974,23/06/2024 20:00,37.0,98,191.4,30.0,80.0,58.0,34.00,20.0
20975,23/06/2024 21:00,35.6,98,233.6,39.7,60.7,58.0,34.33,23.7
20976,23/06/2024 22:00,34.4,99,275.8,49.3,41.3,58.0,34.67,27.3


In [5]:
data = df[['no2']]

In [6]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [7]:
scaled = scaled.reshape((20978,))

In [8]:
scaled.shape

(20978,)

In [9]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [10]:
def train_test_split(X, y, test_size=0.2):
    split_idx = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    return X_train, X_test, y_train, y_test

In [11]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

X_max = X_train.max()
X_min = X_train.min()

In [12]:
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [13]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0212 - root_mean_squared_error: 0.1439 - val_loss: 0.0048 - val_root_mean_squared_error: 0.0692 - learning_rate: 0.0010
Epoch 2/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0149 - root_mean_squared_error: 0.1220 - val_loss: 0.0046 - val_root_mean_squared_error: 0.0680 - learning_rate: 0.0010
Epoch 3/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0148 - root_mean_squared_error: 0.1217 - val_loss: 0.0045 - val_root_mean_squared_error: 0.0671 - learning_rate: 0.0010
Epoch 4/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0147 - root_mean_squared_error: 0.1210 - val_loss: 0.0045 - val_root_mean_squared_error: 0.0671 - learning_rate: 0.0010
Epoch 5/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0145 - root_mean_squared_error: 0.1205 - val_loss: 0.0045 - val_root_mean_squared_error: 0.0671 - learning_rate: 0.0010
Epoch 6/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0143 - root_mean_squared_err

In [14]:
forecast = model.predict(X_test)

130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [15]:
poll = np.array(df["no2"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 1.6601526


In [18]:
# Calculate MAPE
mape = np.mean(np.abs((y_test_true - testPredict) / y_test_true)) * 100
print("Test MAPE =", mape)

Test MAPE = 4.469982162117958


In [19]:
# Calculate accuracy as 1 - MAPE
accuracy = 100 - mape
print("Accuracy =", accuracy, "%")

Accuracy = 95.53001783788204 %


In [20]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(y_test_true, testPredict)
print("Test MAE =", mae)

Test MAE = 1.2170553


In [21]:
forecast

array([[0.3710157 , 0.34192315, 0.31120124, ..., 0.4047989 , 0.41207904,
        0.40942085],
       [0.3342556 , 0.30194515, 0.26915357, ..., 0.40800318, 0.40626663,
        0.41410595],
       [0.29961056, 0.2710051 , 0.24222723, ..., 0.4059026 , 0.41090944,
        0.38831222],
       ...,
       [0.22851238, 0.248283  , 0.27695808, ..., 0.10000403, 0.17374986,
        0.24725172],
       [0.22285765, 0.24914005, 0.26790625, ..., 0.17100903, 0.24052866,
        0.27543777],
       [0.20494929, 0.21990621, 0.23871854, ..., 0.24456888, 0.26753277,
        0.29864514]], dtype=float32)

In [22]:
testPredict

array([[32.05042 , 31.444637, 30.80493 , ..., 32.753872, 32.905464,
        32.850113],
       [31.28498 , 30.612194, 29.92939 , ..., 32.820595, 32.78443 ,
        32.94767 ],
       [30.563581, 29.967943, 29.368715, ..., 32.776855, 32.881107,
        32.410576],
       ...,
       [29.083138, 29.494812, 30.0919  , ..., 26.407265, 27.94284 ,
        29.473337],
       [28.965391, 29.512657, 29.903418, ..., 27.88577 , 29.333347,
        30.060244],
       [28.592493, 28.903934, 29.295654, ..., 29.417475, 29.895641,
        30.54348 ]], dtype=float32)

In [23]:
y_test_true

array([[31.911436, 31.152786, 30.410276, ..., 33.20276 , 33.31575 ,
        33.412598],
       [31.152786, 30.410276, 29.651625, ..., 33.31575 , 33.412598,
        33.52559 ],
       [30.410276, 29.651625, 29.49021 , ..., 33.412598, 33.52559 ,
        32.5571  ],
       ...,
       [28.473295, 28.40873 , 28.360304, ..., 26.261908, 27.714643,
        29.16738 ],
       [28.40873 , 28.360304, 29.215805, ..., 27.714643, 29.16738 ,
        30.733107],
       [28.360304, 29.215805, 30.087446, ..., 29.16738 , 30.733107,
        32.282692]], dtype=float32)

In [24]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [25]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 0.3820
RMSE for segment 2: 0.6573
RMSE for segment 3: 0.8882
RMSE for segment 4: 1.0518
RMSE for segment 5: 1.1768
RMSE for segment 6: 1.2632
RMSE for segment 7: 1.3145
RMSE for segment 8: 1.3503
RMSE for segment 9: 1.3740
RMSE for segment 10: 1.3914
RMSE for segment 11: 1.4010
RMSE for segment 12: 1.4084
RMSE for segment 13: 1.4149
RMSE for segment 14: 1.4227
RMSE for segment 15: 1.4270
RMSE for segment 16: 1.4301
RMSE for segment 17: 1.4324
RMSE for segment 18: 1.4309
RMSE for segment 19: 1.4291
RMSE for segment 20: 1.4305
RMSE for segment 21: 1.4266
RMSE for segment 22: 1.4240
RMSE for segment 23: 1.4181
RMSE for segment 24: 1.4203
RMSE for segment 25: 1.4288
RMSE for segment 26: 1.4458
RMSE for segment 27: 1.4662
RMSE for segment 28: 1.4896
RMSE for segment 29: 1.5085
RMSE for segment 30: 1.5234
RMSE for segment 31: 1.5362
RMSE for segment 32: 1.5483
RMSE for segment 33: 1.5566
RMSE for segment 34: 1.5614
RMSE for segment 35: 1.5666
RMSE for segment 36: 1.5712
R

In [26]:
model.save("no2_1dcnn.keras")